# 02. CNN으로 MNIST 분류하기

## 1. 모델 이해하기
### 1. 첫번째 표기 방법
합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고, 맥스풀링(nn.MaxPoold2d)은 풀링 층으로 별도로 명명합니다.
### 2. 두번째 표기 방법
합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층으로 봅니다.

**1번 레이어 : 합성곱층(Convolutional layer)**
  
  
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
  
  
맥스풀링(kernel_size=2, stride=2))

**2번 레이어 : 합성곱층(Convolutional layer)**  

합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU  

맥스풀링(kernel_size=2, stride=2))

**3번 레이어 : 전결합층(Fully-Connected layer)**  

특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136  

전결합층(뉴런 10개) + 활성화 함수 Softmax

## 2. 모델 구현하기

In [20]:
import torch
import torch.nn as nn

# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

conv1 = nn.Conv2d(1, 32, 3, padding=1) 
# 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1입니다.
print(conv1)

conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
# 32채널 짜리를 입력받아서 64채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1입니다.
print(conv2)

pool = nn.MaxPool2d(2)
# 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당값으로 지정
print(pool)

텐서의 크기 : torch.Size([1, 1, 28, 28])
Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


## 3. 구현체를 연결하여 모델 만들기

In [23]:
out = conv1(inputs)
# print(out)
print(out.shape)

out = pool(out)
print(out.shape)

out = conv2(out)
print(out.shape)

out = pool(out)
print(out.shape)

out.size(0)

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


1

In [26]:
# tensor를 펼치는 작업
out = out.view(out.size(0), -1) 
print(out.shape)

fc = nn.Linear(3136, 10) # input_dim = 3,136, output_dim = 10

# 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환합니다.
out = fc(out)
print(out.shape)

torch.Size([1, 3136])
torch.Size([1, 10])


## 3. CNN으로 MNIST 분류하기

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)